<a id="at_the_top"></a>
# Your PERSONAL Reflective Essay Assistant
Authors: Philippe and Jerome (2024-11-28)

This notebook is designed as a tool to assist students in the process of writing their reflective essay.

**Features:**
1. Guided Reflection: Provides structured support to help students articulate their thoughts and experiences effectively.
2. Instant Personalized Feedback: Offers detailed, actionable suggestions to enhance essay quality, focusing on clarity, depth, and critical thinking.
3. Support for Challenging Areas: Helps students overcome difficulties such as understanding expectations, organizing their thoughts, and developing critical insights.

To get started, run all cells of this notebook.


### Code Logic and functions:

In [7]:
# Load libraries and functions
from IPython.display import display
import ipywidgets as widgets
from openai import OpenAI

# The client will be instantiated with the first call to the API.
openai_client = None

# Keep a history of all messages
list_history_messages = []
list_message =  []

# Call ChatGPT Model
def get_completion_from_messages(list_message, model="gpt-4o", api_key=None, temperature=0.25):
    try:
        # Setup the openai client if not yet done.
        global openai_client
        if not openai_client:
            openai_client = OpenAI(api_key=api_key)

        # If this is the first prompt, add the system message prior the user message.
        if len(list_history_messages) == 0:
            list_history_messages.append({'role':'system', 'content': get_system_message()})

        # Extend the history with the new messages and pass it to the model
        list_history_messages.extend(list_message)

        # Call the OpenAI API
        response = openai_client.chat.completions.create(
            model=model,
            messages=list_history_messages,
            temperature=temperature,
            # Further refinement of the hyperparamters, which shall not be
            # modified from outside.
            max_tokens=512,
            top_p=1,
            frequency_penalty=0.5,
            presence_penalty=0
        )
        response_message = response.choices[0].message.content

    except Exception as err:
        response_message = f"""Something went wrong with the API Call...
        Probably there is an issue with your API key. <br>
        Please read the error message and rerun the notebook. Error message:<br>{err}
        """

    finally:
        # Add the response message to the history
        list_history_messages.extend([{'role': 'assistant', 'content': response_message}])

    return response_message

# New session with the assistant
def reset_assistant():
    global list_history_messages
    global list_message
    # Remove all messages except 'system' messages which is equivalent to a new chat session
    list_history_messages = [element for element in list_history_messages if element["role"] == "system"]

    # unset the list of messages
    list_message = []

    return

def get_system_message():
    system_prompt = """
You are a university teacher specializing in reflective writing, aiming to guide students in both improving their writing skills and gaining a deeper understanding of their experiences. You will offer tailored feedback based on Gibbs' Reflective Cycle, providing direct and actionable advice without rewriting content or strictly enforcing models.

Focus on two primary cases:
1. **Providing personalized feedback to students on their reflective writing submissions.**
2. **Supporting students struggling with starting or structuring reflective essays.**

Your feedback and guidance should be effective, concise, and oriented towards helping students self-develop their reflective abilities.

# Gibbs' Reflective Cycle Feedback Criteria
Use the following components of Gibbs' Reflective Cycle to assess reflective writing:

- **Description**: Assess if the student has described the event or experience clearly. If lacking, briefly indicate which areas need more detail.
- **Feelings**: Has the student explored their emotions adequately? If not, offer prompts that encourage further exploration.
- **Evaluation**: Evaluate if both the positive and negative elements are discussed. Offer suggestions where balance is needed.
- **Analysis**: Determine if the causes and underlying reasons are addressed. Prompt the student to explore deeper insights.
- **Conclusion**: Ensure the student has articulated what they've learned. Prompt them to identify clear lessons.
- **Action Plan**: Check if future actions or strategies are well stated. If missing, suggest they consider specific actions.

# Your assessments should follow these guidelines:
- **Keep Feedback Concise**: Avoid lengthy explanations and instead focus on brief, practical advice.
- **Be Direct but Polite**: Avoid casual comments such as greetings or "thank you for sharing," and stay focused on improvement.
- **Ask Open-Ended Questions**: When prompting further reflection, use questions to stimulate deeper thinking.
- **Use Clear Examples**: Cite specific parts of their writing to clarify your feedback.
- **Consider Gibbs’ Cycle as a Framework**: Use Gibbs' Reflective Cycle to guide feedback, but do not force full adherence to this structure. Respect the student's existing achievement and voice.
- **Rating Scale for each Stage**:
  - ++ (Excellent): The stage is thoroughly addressed with insightful reflection.
  - + (Good): The stage is adequately addressed but could be expanded.
  - - (Needs Improvement): The stage is mentioned but lacks depth.
  - -- (Missing/Inadequate): The stage is missing or not addressed properly.

# Providing Feedback: Differentiating Two Cases
## Case 1: Students Requesting Instant Personalized Feedback
Offer the following types of feedback for students who have submitted their reflective writing:

1. **### General Feedback**: Provide a brief (3-4 sentences) general comment on overall writing quality; summarize strengths and general areas for improvement.
2. **### Gibbs’ Reflective Cycle Feedback**: Comment briefly on each section of the cycle. Address whether the points were adequately covered, providing clear guidance for improvement.
3. **### Rating Overview**: Provide a rating for each Gibbs’ Reflective Cycle stage using the scale (++, +, -, --) in a list format.
   - **Description**: [++/+/--/-]
   - **Feelings**: [++/+/--/-]
   - **Evaluation**: [++/+/--/-]
   - **Analysis**: [++/+/--/-]
   - **Conclusion**: [++/+/--/-]
   - **Action Plan**: [++/+/--/-]
4. **### Actionable Points**: Suggest 3 specific, practical steps ("quick wins") for improving the reflective writing, highlighting specific areas or sentences where changes can be easily implemented.

## Case 2: Students Needing Guidance in Reflective Writing
For students struggling to start or organize their writing, the output should consist of:

1. **### General Guidance**: A succinct paragraph (5-6 sentences) explaining how to begin the reflective essay. Offer concise tips on structuring a reflective essay. Include how to use Gibbs' Reflective Cycle as a framework without adhering rigidly, emphasizing key aspects like focusing on description, emotions, and lessons learned.

# Examples
### Example 1 (Providing Feedback):
**Student Input**: "In my internship, I felt overwhelmed at first, but soon I improved and started to enjoy the tasks."
**General Feedback**: You provided a good overview of the emotional journey during your internship. To improve, expand on specific tasks that caused the shift.
**Gibbs' Reflective Cycle Feedback**:
  - **Description**: Mentioned the internship experience but could benefit from more details about an individual task.
  - **Feelings**: Emotions are noted, though reflecting on why you were overwhelmed could deepen insight.
  - **Evaluation**: Lacks evaluation of what exactly improved—consider specifics.
  - **Analysis**: Briefly mentioned, expand on why you transitioned from feeling overwhelmed to confident.
  - **Conclusion**: Missing. What did you learn about managing overwhelming situations?
  - **Action Plan**: Not stated. Consider focusing on how these lessons can aid future work environments.
**Rating Overview**:
  - **Description**: +
  - **Feelings**: ++
  - **Evaluation**: -
  - **Analysis**: --
  - **Conclusion**: +
  - **Action Plan**: --
**Actionable Points**:
  1. Describe a specific instance that led to feeling overwhelmed.
  2. Reflect on why your feelings changed over time.
  3. Conclude by articulating one lesson you learned.

### Example 2 (Offering General Guidance):
**General Guidance**: To begin a reflective essay, start by clearly describing the event that you're reflecting on—what happened, where and who was involved. Then, explore your emotions during this process. What specifically did you find challenging or rewarding? Apply Gibbs' Reflective Cycle to guide your reflection but allow flexibility in structuring it. Try to provide insight into what led to these experiences, conclude with lessons learned, and finally, plan what actions you could take in similar future situations.

# Final Remarks
- Avoid over-editing or changing the student's unique perspective. Focus on guiding rather than rewriting.
- Ensure feedback is respectful of each student's existing reflections while challenging them to think deeper.
- Always include practical examples or prompts tailored to each individual's writing, encouraging self-discovery.
- The output should be in plain HTML format, your output is embedded in <html></html>
"""

    return system_prompt


def get_html_conversation(list_history_messages):
    # Prepare the HTML output for the user.
    html_template = """
    <!DOCTYPE html>
    <html>
      <head>
        <style>
          .chat-container {{
              display: flex;
              flex-direction: column;
              .user, .assistant {{
                  margin-top: 1.5rem;
                  padding: 0.5rem;
                  border-radius: 0.5rem;
              }}
              .user {{
                  margin-left: auto;
                  border: solid 1px blue;
                  max-width: 55%;
              }}
              .assistant {{
                  margin-right: auto;
                  border: solid 1px red;
                  max-width: 55%;
              }}
            }}
        </style>
      </head>
      <body>

        <div class="chat-container">
          <!-- Test Data in a HTML comment
          <div class="user">Hi! How are you?</div>
          <div class="assistant">Hello! Great, and yourself?</div>
          -->
            {div_messages}
        </div>

      </body>
    </html>
"""

    # Take the history of all messages and create the structure for the conversion,
    # organized by the role of the message
    div_prepared_messages = ""

    # We will only consider "user" or "assistant" messages
    # (Hide "system" messages from the user)
    for message in list_history_messages:
        message_current = message["content"]

        if message["role"] == "user":
          div_prepared_messages += f"<div class='user'>{message_current}</div>"

        elif message["role"] == "assistant":
          div_prepared_messages += f"<div class='assistant'>{message_current}</div>"

        else:
            # Skip "system" messages
            pass

    # Extend the html template by the messages from the history
    html_prepared = html_template.format(div_messages="".join(div_prepared_messages))
    return html_prepared


In [8]:
# Define iteractive elements for Juypter
text_user_input = widgets.Textarea(
    value="", disabled=False,
    placeholder="Ask for advice from the Reflective Essay Assistant.",
    layout=widgets.Layout(width="90%", height="200px", overflow_y="auto")
)

text_api_key = widgets.Password(
    value="", disabled=False,
    placeholder="Insert your OpenAI Key here",
    layout=widgets.Layout(width="45%")
)

button_chat = button = widgets.Button(
    description="Submit", disabled=False,
    button_style="info", icon="check"
)

button_clear = button = widgets.Button(
    description="Clear Chat", disabled=False,
    button_style="warning", icon="trash"
)

output_conversation = widgets.Output() # layout=widgets.Layout(max_height="500px", overflow_y="auto"))
output_user_warning = widgets.Output()


def on_clicked_button_chat(b):
    # If no user input was provided, inform user to provide input
    str_user_input = text_user_input.value
    if str_user_input == "":
        # Clear prior warning massages, if any
        output_user_warning.clear_output()

        # Display a message to the user
        text_user_input.placeholder = "Ask for advice from the Reflective Essay Assistant."
        with output_user_warning:
            print("No input provided... Ask me something!")

    else:
        # Clear prior warning massages, if any; and
        # Clear the input field to be ready for the next message
        # Also, hide the input field for the api key...
        output_user_warning.clear_output()
        text_user_input.value = ""
        text_api_key.layout.display = "none"

        # Replace the default placeholder to indicate prompt submission
        text_user_input.placeholder = "Thinking..."

        # Call the API
        list_message = [{'role':'user', 'content': str_user_input}]

        # Interract with the assistant;
        # response is stored in list_history_messages
        get_completion_from_messages(list_message=list_message, api_key=text_api_key.value)

        output_html = widgets.HTML(
            value=get_html_conversation(list_history_messages=list_history_messages),
            placeholder="Some HTML",
        )

        # Display conversation and replace old one
        output_conversation.clear_output()
        with output_conversation:
            display(output_html)

        # Notify user the the response is ready
        text_placeholder = "Done! See my feedback below. Let me know if I can further help you with your essay."
        text_user_input.placeholder = text_placeholder

    return

button_chat.on_click(on_clicked_button_chat)

def on_clicked_button_clear(b):
    # Clear chat history, new chat
    reset_assistant()

    # Clear the output fields and
    # Clear the input field to be ready for the next message
    output_conversation.clear_output()
    output_user_warning.clear_output()
    text_user_input.value = ""
    text_user_input.placeholder = "Ask for advice from the Reflective Essay Assistant."

    # Confirmation for the user
    with output_user_warning:
        print("History cleared, new chat session started.")

    return

button_clear.on_click(on_clicked_button_clear)

# Re-arrange elements for the user
allignment_horizontal = widgets.HBox([button_chat, button_clear])
allignment_vertical = widgets.VBox([text_api_key, text_user_input, allignment_horizontal, output_user_warning, output_conversation])
user_gui = allignment_vertical

### Display Assistant to the student:
[Jump to the bottom](#scrollTo=01751366-5c41-46c5-be8d-7983bd7ff1ca&line=1&uniqifier=1)


In [9]:
# Make the interaction visible
display(user_gui)

[Jump back to the top](#scrollTo=57510507-c424-449e-8a8c-9a184c3bf1e1)

# <span style="color:white">The very bottom of this document</span>